In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

from tensorflow.keras.applications import ResNet50, DenseNet121, VGG16, InceptionResNetV2
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

Using TensorFlow backend.


In [2]:
train = pd.read_csv('/kaggle/input/classification-of-images/dataset/train.csv')
test = pd.read_csv('/kaggle/input/classification-of-images/dataset/test.csv')
train.head()

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [3]:
Class_map = {'Food': 0, 'Attire': 1, 'Decorationandsignage': 2, 'misc': 3}
inverse_map = {0: 'Food', 1: 'Attire', 2: 'Decorationandsignage', 3: 'misc'}
train['Class'] = train['Class'].map(Class_map)

In [ ]:
train_img = []
train_label = []
j = 0
path = '/kaggle/input/classification-of-images/dataset/Train Images'
for i in tqdm(train['Image']):
    final_path = os.path.join(path, i)
    img = cv2.imread(final_path)
    img = cv2.resize(img, (150, 150))
    img = img.astype('float32')
    train_img.append(img)
    train_label.append(train['Class'][j])
    j=j+1

 98%|█████████▊| 5841/5983 [00:14<00:00, 405.68it/s]

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(train_img)

In [ ]:
test_img = []
path = '/kaggle/input/classification-of-images/dataset/Test Images'
for i in tqdm(test['Image']):
    final_path = os.path.join(path,i)
    img = cv2.imread(final_path)
    img = cv2.resize(img,(150,150))
    img = img.astype('float32')
    test_img.append(img)

In [ ]:
train_img = np.array(train_img)
test_img = np.array(test_img)
train_label = np.array(train_label)
print(train_img.shape)
print(test_img.shape)
print(train_label.shape)

In [ ]:
base_models = [VGG16(include_top=False, weights='imagenet',input_shape=(150,150,3), pooling='avg'),
              ResNet50(include_top=False, weights='imagenet',input_shape=(150,150,3), pooling='avg'),
              DenseNet121(include_top=False, weights='imagenet',input_shape=(150,150,3), pooling='avg'),
              InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(150,150,3), pooling='avg')]

In [ ]:
def create_model(base_model, random_hidden_units):    
    base_model.trainable = False
    model = Sequential()
    model.add(base_model)
    model.add(Dense(random_hidden_units,activation='relu'))
    model.add(Dense(4,activation='softmax'))
    return model

In [ ]:
n_train, x, y, z = train_img.shape
n_test, x, y, z = test_img.shape
numFeatures = x * y * z

In [ ]:
train_labels = to_categorical(train_label,4)

In [ ]:
np.random.seed(seed=1997)
# Number of estimators
n_estimators = 4
# Proporition of samples to use to train each training
max_samples = 0.7
max_samples *= n_train
max_samples = int(max_samples)

In [ ]:
models = list()
# random = np.random.randint(50, 256, size=n_estimators)
random = [256]*4

for i in range(n_estimators):    
    # Model
    model = create_model(base_models[i], random[i])    
    
    reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
                                         factor=0.1,
                                         patience=2,
                                         cooldown=2,
                                         min_lr=0.00001,
                                         verbose=1)

    callbacks = [reduce_learning_rate]

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])    
    
    # Store model
    models.append(model)

In [ ]:
histories = []

for i in range(n_estimators):
    # Train each model on a bag of the training data
    train_idx = np.random.choice(len(train_img),
                                 size = max_samples)
    
    histories.append(models[i].fit_generator(datagen.flow(train_img[train_idx], 
                                                          train_labels[train_idx], 
                                                          batch_size=32),
                                             epochs=10,
                                             callbacks=callbacks))    

In [ ]:
predictions = []
for i in range(n_estimators):
    predictions.append(models[i].predict(test_img))
    
predictions = np.array(predictions)
predictions = predictions.sum(axis = 0)
pred_labels = predictions.argmax(axis=1)

class_label = [inverse_map[x] for x in pred_labels]
print(class_label[:3])
submission = pd.DataFrame({'Image': test.Image, 'Class': class_label })
submission.head(10)
submission.to_csv('submission.csv', index=False)